<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as krs
import tensorflow as tsf
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.text import Tokenizer
from keras.layers import Conv1D,Conv2D,MaxPooling2D
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import itertools
from google.colab import files
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack

stopwordsList = stopwords.words('english')

#url1 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-bagofwords.csv'
#url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-bagofwords.csv'
url1 = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train = pd.read_csv(url1)
test = pd.read_csv(url2)

train.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
# Se eliminan stop words

ps = PorterStemmer()

def dropStopwords(texts):
   results=[] 
   for i in range(0, len(texts)):
      text = ' '.join([word for word in texts[i].split() if word not in stopwordsList])
      results.append(text)
   return results
   
train['text'] = dropStopwords(train['text'])

In [18]:
tokenizer = Tokenizer()
classifier=MultinomialNB(1)
TfIdf=TfidfVectorizer()

In [19]:
def add_feature(data, feature):
  
    return hstack([data, csr_matrix(feature).T], 'csr')


In [20]:
# Se crea el modelo

data_target = train[['target']]
data_train=train.drop(['id', 'keyword', 'location', 'target'],axis = 1)
data_test=test.drop(['keyword', 'location'],axis = 1)

#scaler = StandardScaler()
#scaler.fit(data_train)

#data_train=scaler.transform(data_train)
#data_test=scaler.transform(data_test)
tokenizer.fit_on_texts(data_train)

cv_train = CountVectorizer(analyzer='word',max_features=5660,ngram_range=(1,2),binary=True)

data_train_v = cv_train.fit_transform(data_train['text']).toarray()

#data_train_v_f = add_feature(data_train_v, data_train['text'].str.len())
#data_train_v = add_feature(data_train_v, data_train['text'].str.replace(r'\D+', '').str.len())
#data_train_v = add_feature(data_train_v, data_train['text'].str.replace(r'\w+', '').str.len())
data_train_v.shape
data_train_v

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
# Se divide el set en train y validation

x_train, x_validation, y_train, y_validation = model_selection.train_test_split(data_train_v, data_target, test_size=0.3, random_state=0)

cv_train.get_params()
cv_train_DF = pd.DataFrame(x_train, columns=cv_train.get_feature_names())


In [22]:

#model=krs.Sequential()
#model.add(krs.layers.embeddings)
#model.add(Conv1D(128, kernel_size=(3),activation='relu'))
#model.add(krs.layers.Dense(8, input_dim=2, activation='relu'))
#model.add(krs.layers.Dense(8, activation='relu'))
#model.add(krs.layers.Dense(2, activation='softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x_train, y_train, epochs=12, batch_size=128,validation_data=(x_validation,y_validation))

In [23]:
# Se entrena y predice

classifier.fit(x_train, y_train)
prediction = classifier.predict(x_validation)
prediction

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 1, ..., 1, 1, 1])

In [24]:
# Precisión (Hasta 3% de diferencia)

accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

0.8117338003502627

In [25]:
#data_target = countV.fit_transform(data_target['target']).toarray()
#data_target.shape
#cv_train2 = CountVectorizer(max_features=5000,ngram_range=(1,2))
#cv_train.get_params()
#cv_train2_DF = pd.DataFrame(data_train, columns=cv_train.get_feature_names())
classifier.fit(data_train_v, data_target)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [26]:
# Se predice con test luego de reentrenar con el set completo

cv_test = CountVectorizer(vocabulary=cv_train.get_feature_names(),analyzer='word',max_features=5660,ngram_range=(1,2),binary=True)

x_test = cv_test.transform(data_test['text']).toarray()
x_test.shape

cv_test.get_params()
cv_test_DF = pd.DataFrame(x_test, columns=cv_test.get_feature_names())

prediction2 = classifier.predict(x_test)


In [27]:
x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [28]:
# Se genera submit

data_test['target'] = prediction2
export = pd.DataFrame()
export['id'] = data_test['id']
export['target'] = data_test['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>